In [2]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import json

In [1]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 6 not upgraded.
Need to get 2564 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.1 [916 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 2564 kB in 1s (2875 kB/s)       
Selecting previously unselected package 

In [3]:
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-san

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-san
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 4226 kB of archives.
After this operation, 12.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-san all 1:4.00~git30-7274cfa-1 [4226 kB]
Fetched 4226 kB in 0s (32.9 MB/s)          
Selecting previously unselected package tesseract-ocr-san.
(Reading database ... 108210 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-san_1%3a4.00~git30-7274cfa-1_all.deb ...
Unpacking tesseract-ocr-san (1:4.00~git30-7274cfa-1) ...
Setting up tesseract-ocr-s

In [12]:
pdf_path = '/kaggle/input/sanskrit-text/Sanskrit_Text (1).pdf'

# Convert the PDF pages to images
pages = convert_from_path(pdf_path)

In [5]:
!tesseract --list-langs

List of available languages (3):
eng
osd
san


In [6]:
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [7]:
boxes = {}

In [14]:
# Loop through each page image
for i, page in enumerate(pages):
    # Extract the text and bounding box information from the image
    data = pytesseract.image_to_data(page, output_type=pytesseract.Output.DICT)
    
    # Initialize variables to store the current line and its bounding box coordinates
    line = ''
    x_min = float('inf')
    y_min = float('inf')
    x_max = 0
    y_max = 0
    
    # Loop through each text line
    for j in range(len(data['text'])):
        # Check if the text is not empty
        if data['text'][j].strip() != '':
            # Add the text to the current line
            line += data['text'][j] + ' '
            
            # Update the bounding box coordinates
            x_min = min(x_min, data['left'][j])
            y_min = min(y_min, data['top'][j])
            x_max = max(x_max, data['left'][j] + data['width'][j])
            y_max = max(y_max, data['top'][j] + data['height'][j])
        else:
            # Check if the current line is not empty
            if line.strip() != '':
                # Add the bounding box coordinates to the dictionary
                box_name = f'box{i}_{j}'
                boxes[box_name] = {
                    'top_left': [x_min, y_min],
                    'top_right': [x_max, y_min],
                    'bottom_left': [x_min, y_max],
                    'bottom_right': [x_max, y_max]
                }
                
                # Crop the image to the bounding box and save it as a separate JPG file
                line_image = page.crop((x_min, y_min, x_max, y_max))
                line_image.save(f'{box_name}.jpg')
                
                # Reset the current line and its bounding box coordinates
                line = ''
                x_min = float('inf')
                y_min = float('inf')
                x_max = 0
                y_max = 0
# Save the bounding box coordinates to a JSON file
with open('boxes.json', 'w') as f:
    json.dump(boxes, f)

In [18]:
import json
from IPython.display import FileLink

# Set the name of the JSON file
json_filename = 'boxes.json'

# Save the bounding box coordinates to a JSON file
with open(json_filename, 'w') as f:
    json.dump(boxes, f)

# Create a download link for the JSON file
FileLink(json_filename)

/kaggle/working/boxes.json

In [15]:
import os
import shutil
from IPython.display import FileLink

# Set the name of the new directory and the compressed archive
dir_name = 'images'
archive_name = 'images.zip'

# Create the new directory
os.makedirs(dir_name, exist_ok=True)

# Move all image files from the current working directory to the new directory
for filename in os.listdir():
    if filename.endswith('.jpg'):
        shutil.move(filename, os.path.join(dir_name, filename))

# Create a compressed archive of the new directory
shutil.make_archive(archive_name, 'zip', dir_name)

# Create a download link for the compressed archive
FileLink(f'{archive_name}.zip')

/kaggle/working/images.zip.zip